In [1]:
%load_ext autoreload
%autoreload 2
import torch
from Feat2Annot import *
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from torcheval import metrics

In [2]:
from util import prepare_dataset

dataset = prepare_dataset("./data", 15)
print(dataset.get_annot_class())

689459it [00:00, 2059662.13it/s]                            


{'class': 7, 'weight': array([0.00265764, 0.04788601, 0.01087034, 0.00688993, 0.01129333,
       0.91583195, 0.0045708 ])}


In [3]:
batch_size = 32
generator = torch.Generator().manual_seed(42)
train_data, val_data = torch.utils.data.random_split(dataset, (0.9, 0.1), generator)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=False, batch_size=1)

In [4]:
model = Feat2AnnotModel(
    input_size=200, hidden_size=1024, target_class=dataset.get_annot_class()
)

In [ ]:
### Initial model score
metric = metrics.MulticlassAccuracy()
metric.reset()
model.eval()
for source_feature, tgt_annot in tqdm(val_loader):
    annot_hypothesis = model.beam_search(source_feature, 1)
    annot_hat = torch.tensor(annot_hypothesis[0].value, dtype=torch.int64)
    metric.update(annot_hat, tgt_annot.squeeze(0))
init_valid_metric = metric.compute()

In [6]:
### Run one episode of training
metric = metrics.MulticlassAccuracy()
num_epoch = 1
training_losses = []
val_losses = []
val_metric = []
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
for epoch in range(num_epoch):
    metric.reset()
    model.train()
    for xbatch, ybatch in tqdm(train_loader):
        output = model(xbatch, ybatch)
        loss = -output
        batch_loss = loss.sum()
        loss = batch_loss / batch_size
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    # with torch.no_grad():
    #     for xval_batch, yval_batch in tqdm(val_loader):
    #         output = model(xval_batch, yval_batch)
    #         val_loss = -output.sum() / batch_size
    #         annot_hypothesis = model.beam_search(xval_batch, 1)
    #         annot_hat = torch.tensor(annot_hypothesis[0].value, dtype=torch.int64)
    #         metric.update(annot_hat, yval_batch.squeeze(0))
    #     val_metric.append(metric.compute())

    # print(
    #     f"epoch {epoch+1}/{num_epoch}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}"
    # )
    training_losses.append(loss.item())
    # val_losses.append(val_loss.item())

100%|██████████| 802/802 [04:35<00:00,  2.91it/s]


In [42]:
# Example tensor with 0s and 1s
tensor = torch.tensor(
    [
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 0, 1, 0, 1],
    ]
)

# Define a convolutional kernel to perform the bleeding operation
kernel = torch.tensor([[1, 1, 1]])

# Perform 1D convolution with the kernel along each row of the tensor
result = torch.nn.functional.conv1d(
    tensor.unsqueeze(1).float(), kernel.unsqueeze(0).float(), padding=1
).squeeze(0)

# Convert the result to binary tensor by thresholding
result_binary = (result > 0).int()

print("Original Tensor:")
print(tensor)
print("\nBleeding Result:")
print(result_binary)

Original Tensor:
tensor([[0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 0, 1, 0, 1]])

Bleeding Result:
tensor([[[0, 1, 1, 1, 0, 0, 0, 0, 0]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 0]],

        [[1, 1, 1, 1, 0, 1, 1, 1, 1]]], dtype=torch.int32)


In [ ]:
# print(f"naive model val accuracy {init_valid_metric}")
print(f"after one episode of training {val_metric[-1]}")

In [15]:
model.eval()
new_val_loader = DataLoader(
    dataset=val_loader.dataset,
    batch_size=1,
    shuffle=True,
)
dataiter = iter(new_val_loader)
source_feature, target_seq = next(dataiter)
annot_hypothesis = model.beam_search(source_feature, 1)
annot_hat = torch.tensor(annot_hypothesis[0].value, dtype=torch.int64)
print(f"target {target_seq}")
print(f"predicted {annot_hat}")

target tensor([[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0]])
predicted tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
## load params
# load model
params = torch.load("model.bin", map_location=lambda storage, loc: storage)
model.load_state_dict(params["state_dict"])

In [ ]:
a = torch.tensor(
    [
        [1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0],
    ]
)
b, c, d = exponential_weight(a, 0.25)
from matplotlib import pyplot

fig, axes = pyplot.subplots(1, 4)
axes[0].plot(b[1, :])
axes[1].plot(a[1, :])
axes[2].plot(c[1, :])
axes[2].plot(d[1, :])
print(c[0, :], d[1, :])